# Model Context Protocol (MCP) Labs - PART 1 (Google Colab)

## 🚀 Running on Google Colab

This notebook is optimized for **Google Colab** with:
- ✅ Node.js installation in Colab
- ✅ MCP server setup without localhost
- ✅ File upload for credentials
- ✅ Environment variables via Colab secrets
- ✅ Persistent storage via Google Drive (optional)

### Part 1 Labs:
- **Lab 0**: Complete MCP Setup for Google Colab
- **Lab 1**: MCP Architecture
- **Lab 2**: Filesystem MCP
- **Lab 3**: Gmail MCP (cygenaidemo@gmail.com)
- **Lab 4**: Google Calendar MCP

### Use Cases:
1. **HR Recruitment** - Interview invitations, scheduling
2. **AirBnb Management** - Booking confirmations

### 📧 Sender Email
**cygenaidemo@gmail.com**

---

# LAB 0: Complete MCP Setup for Google Colab

## Part A: Google Cloud Project Setup (Do this FIRST)

### Step 1: Create Google Cloud Project

Open this in a new tab: [Google Cloud Console](https://console.cloud.google.com/)

1. Click **"Select a project"** → **"New Project"**
2. Project Name: `mcp-colab-labs`
3. Click **"Create"**

### Step 2: Enable APIs

In Google Cloud Console:

1. Go to **"APIs & Services"** → **"Library"**
2. Search and enable:
   - ✅ **Gmail API**
   - ✅ **Google Calendar API**

### Step 3: Create OAuth 2.0 Credentials

1. Go to **"APIs & Services"** → **"Credentials"**
2. Click **"Create Credentials"** → **"OAuth client ID"**

3. **Configure OAuth consent screen** (if first time):
   - User Type: **External**
   - App name: `MCP Colab Labs`
   - User support email: `cygenaidemo@gmail.com`
   - Developer email: `cygenaidemo@gmail.com`
   - Click **"Save and Continue"**
   
   **Scopes** (Add these):
   - `https://www.googleapis.com/auth/gmail.send`
   - `https://www.googleapis.com/auth/gmail.readonly`
   - `https://www.googleapis.com/auth/calendar`
   - `https://www.googleapis.com/auth/calendar.events`
   
   **Test users** (Add this):
   - `cygenaidemo@gmail.com`
   
   Click **"Save and Continue"** → **"Back to Dashboard"**

4. Now create **OAuth client ID**:
   - Application type: **Desktop app**
   - Name: `MCP Colab Client`
   - Click **"Create"**

5. **Download JSON** - Click the download icon
   - File will be named: `client_secret_xxxxx.json`
   - **Keep this file safe!**

### ✅ Before proceeding, make sure you have:
- [ ] Google Cloud project created
- [ ] Gmail API enabled
- [ ] Calendar API enabled
- [ ] OAuth credentials JSON downloaded

## Part B: Install Node.js in Google Colab

In [ ]:
%%bash
# Install Node.js 18.x in Colab
echo "📦 Installing Node.js..."

# Install Node.js using NodeSource
curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -
sudo apt-get install -y nodejs

# Verify installation
echo ""
echo "✅ Node.js installed:"
node --version
npm --version

## Part C: Install Python Packages

In [ ]:
# Install Python packages
!pip install -q langchain-openai langgraph langchain-core python-dotenv
!pip install -q langchain-mcp-adapters mcp
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q google-api-python-client

print("✅ Python packages installed")

## Part D: Install MCP Servers

In [ ]:
%%bash
# Install MCP servers globally
echo "📦 Installing MCP servers..."

npm install -g @modelcontextprotocol/server-gmail
npm install -g @modelcontextprotocol/server-google-calendar
npm install -g @modelcontextprotocol/server-filesystem

echo ""
echo "✅ MCP servers installed:"
npm list -g --depth=0 | grep modelcontextprotocol

## Part E: Upload OAuth Credentials

**IMPORTANT**: You'll upload your `client_secret_xxxxx.json` file here.

In [ ]:
from google.colab import files
import json
import os

print("📤 Upload your OAuth credentials JSON file")
print("   (The file you downloaded: client_secret_xxxxx.json)")
print()

uploaded = files.upload()

# Get the uploaded filename
credentials_filename = list(uploaded.keys())[0]

# Create config directory
!mkdir -p /content/mcp_config

# Copy to standard location
!cp "{credentials_filename}" /content/mcp_config/credentials.json

# Verify
with open('/content/mcp_config/credentials.json', 'r') as f:
    creds = json.load(f)
    print(f"\n✅ Credentials uploaded successfully!")
    print(f"   Client ID: {creds['installed']['client_id'][:30]}...")

print(f"\n📁 Credentials saved to: /content/mcp_config/credentials.json")

## Part F: Set Up Environment Variables

### Option 1: Using Colab Secrets (Recommended)

1. Click the **🔑 key icon** in the left sidebar
2. Add a new secret:
   - Name: `OPENAI_API_KEY`
   - Value: Your OpenAI API key
3. Toggle **"Notebook access"** to ON

### Option 2: Direct Input (Less Secure)

In [ ]:
import os
from getpass import getpass

# Try to get from Colab secrets first
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API key loaded from Colab secrets")
except:
    # Fallback to manual input
    print("⚠️ Colab secret not found. Enter manually:")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set environment variable
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Configuration
SENDER_EMAIL = "cygenaidemo@gmail.com"
CREDENTIALS_PATH = "/content/mcp_config/credentials.json"

print(f"\n📧 Sender Email: {SENDER_EMAIL}")
print(f"📁 Credentials: {CREDENTIALS_PATH}")

## Part G: Authenticate Gmail & Calendar

**IMPORTANT**: This will open authentication flows.

For each service:
1. A link will be displayed
2. Click the link and authenticate with **cygenaidemo@gmail.com**
3. Copy the authorization code
4. Paste it back in the input box

In [ ]:
# Gmail Authentication
from google_auth_oauthlib.flow import InstalledAppFlow
import json

print("🔐 Gmail Authentication")
print("="*60)

GMAIL_SCOPES = [
    'https://www.googleapis.com/auth/gmail.send',
    'https://www.googleapis.com/auth/gmail.readonly'
]

# Create flow
gmail_flow = InstalledAppFlow.from_client_secrets_file(
    CREDENTIALS_PATH,
    scopes=GMAIL_SCOPES
)

# Run auth flow (Colab compatible - uses console)
gmail_creds = gmail_flow.run_console()

# Save token
gmail_token_path = '/content/mcp_config/gmail-token.json'
with open(gmail_token_path, 'w') as token:
    token.write(gmail_creds.to_json())

print(f"\n✅ Gmail authenticated!")
print(f"📁 Token saved: {gmail_token_path}")

In [ ]:
# Calendar Authentication
print("\n🔐 Google Calendar Authentication")
print("="*60)

CALENDAR_SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/calendar.events'
]

# Create flow
calendar_flow = InstalledAppFlow.from_client_secrets_file(
    CREDENTIALS_PATH,
    scopes=CALENDAR_SCOPES
)

# Run auth flow
calendar_creds = calendar_flow.run_console()

# Save token
calendar_token_path = '/content/mcp_config/calendar-token.json'
with open(calendar_token_path, 'w') as token:
    token.write(calendar_creds.to_json())

print(f"\n✅ Calendar authenticated!")
print(f"📁 Token saved: {calendar_token_path}")

## Part H: Verify Setup

In [ ]:
# Verification
import os

print("🔍 Setup Verification")
print("="*60)

checks = {
    "Node.js installed": os.system("which node > /dev/null 2>&1") == 0,
    "npm installed": os.system("which npm > /dev/null 2>&1") == 0,
    "Credentials file": os.path.exists(CREDENTIALS_PATH),
    "Gmail token": os.path.exists('/content/mcp_config/gmail-token.json'),
    "Calendar token": os.path.exists('/content/mcp_config/calendar-token.json'),
    "OpenAI API key": os.getenv('OPENAI_API_KEY') is not None
}

for check, passed in checks.items():
    status = "✅" if passed else "❌"
    print(f"{status} {check}")

all_passed = all(checks.values())

if all_passed:
    print("\n🎉 All checks passed! Ready to proceed with labs.")
else:
    print("\n⚠️ Some checks failed. Please review the setup steps.")

## Part I: Create MCP Configuration

In [ ]:
# Create MCP configuration for Colab
import json

mcp_config = {
    "mcpServers": {
        "gmail": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-gmail"],
            "transport": "stdio",
            "env": {
                "GMAIL_CREDENTIALS": "/content/mcp_config/credentials.json",
                "GMAIL_TOKEN": "/content/mcp_config/gmail-token.json",
                "GMAIL_USER": SENDER_EMAIL
            }
        },
        "google_calendar": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-google-calendar"],
            "transport": "stdio",
            "env": {
                "GOOGLE_CALENDAR_CREDENTIALS": "/content/mcp_config/credentials.json",
                "GOOGLE_CALENDAR_TOKEN": "/content/mcp_config/calendar-token.json",
                "GOOGLE_CALENDAR_USER": SENDER_EMAIL
            }
        },
        "filesystem": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-filesystem", "/content/data"],
            "transport": "stdio"
        }
    }
}

# Save configuration
config_path = '/content/mcp_config.json'
with open(config_path, 'w') as f:
    json.dump(mcp_config, f, indent=2)

print("✅ MCP Configuration created")
print(f"📁 Config saved: {config_path}")
print("\n📋 Configuration:")
print(json.dumps(mcp_config, indent=2))

---

## ✅ Lab 0 Complete!

You're now ready to use real Gmail and Calendar MCP servers in Colab.

### What we set up:
- ✅ Node.js in Colab
- ✅ MCP servers installed
- ✅ OAuth credentials configured
- ✅ Gmail authenticated
- ✅ Calendar authenticated
- ✅ MCP configuration file created

### ⚠️ Important Notes for Colab:
1. **Session Persistence**: Colab sessions are temporary. If disconnected, you'll need to:
   - Re-run setup cells (Node.js, MCP servers)
   - Re-upload credentials OR save to Google Drive
   - Authentication tokens should persist if saved

2. **Save to Google Drive** (Optional but Recommended):

In [ ]:
# Optional: Save credentials to Google Drive for persistence
from google.colab import drive

print("💾 Save setup to Google Drive for persistence?")
print("   This allows you to skip setup in future sessions.")
print()

save_to_drive = input("Save to Drive? (y/n): ").lower() == 'y'

if save_to_drive:
    # Mount Google Drive
    drive.mount('/content/drive')
    
    # Create directory
    !mkdir -p /content/drive/MyDrive/mcp_labs_config
    
    # Copy files
    !cp -r /content/mcp_config/* /content/drive/MyDrive/mcp_labs_config/
    !cp /content/mcp_config.json /content/drive/MyDrive/mcp_labs_config/
    
    print("\n✅ Configuration saved to Google Drive")
    print("📁 Location: /content/drive/MyDrive/mcp_labs_config/")
    print("\n💡 Next time, you can restore with:")
    print("   !cp -r /content/drive/MyDrive/mcp_labs_config/* /content/mcp_config/")
else:
    print("\n⏭️ Skipped. You'll need to re-authenticate each session.")

---

# Core Imports and Setup

In [ ]:
# Core imports
import os
import json
import asyncio
from pathlib import Path
from typing import List, Dict, Any
from datetime import datetime, timedelta

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# LangGraph imports
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import create_react_agent

# MCP imports
try:
    from langchain_mcp_adapters.client import MultiServerMCPClient
    from mcp import ClientSession, StdioServerParameters
    MCP_AVAILABLE = True
except ImportError:
    MCP_AVAILABLE = False
    print("⚠️ MCP adapters not available")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✅ Imports complete")
print(f"📧 Sender: {SENDER_EMAIL}")
print(f"🔧 MCP Available: {MCP_AVAILABLE}")

## Sample Data

In [ ]:
# Create data directory
!mkdir -p /content/data/resumes
!mkdir -p /content/data/contracts

# Sample data
HR_DATABASE = {
    "candidates": {
        "CAN001": {
            "name": "Priya Sharma",
            "email": "your-test-email@example.com",  # ← Change for testing
            "position": "Senior Backend Engineer",
            "status": "screening"
        }
    }
}

AIRBNB_DATABASE = {
    "properties": {
        "PROP001": {"name": "Cozy Goa Beach Villa", "price_per_night": 8500}
    },
    "bookings": {
        "BOOK001": {
            "guest_name": "Sarah Johnson",
            "guest_email": "guest-test@example.com",  # ← Change for testing
            "property_id": "PROP001",
            "status": "confirmed"
        }
    }
}

SAMPLE_RESUME = """PRIYA SHARMA
Senior Software Engineer | 6+ years Python/AWS experience
Skills: FastAPI, microservices, Docker, Kubernetes"""

print("✅ Sample data loaded")

---

# LAB 1: MCP Architecture

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         MCP ARCHITECTURE IN GOOGLE COLAB                     ║
╚══════════════════════════════════════════════════════════════╝

COLAB NOTEBOOK
      │
      │ langchain-mcp-adapters
      ▼
┌─────────────────────┐
│MultiServerMCPClient │ ◄─── /content/mcp_config.json
└─────────┬───────────┘
          │
    ┌─────┴─────┬──────────┐
    │           │          │
    ▼           ▼          ▼
┌────────┐ ┌────────┐ ┌────────┐
│ Gmail  │ │Calendar│ │FileSys │
│  MCP   │ │  MCP   │ │  MCP   │
└───┬────┘ └───┬────┘ └───┬────┘
    │          │          │
    ▼          ▼          ▼
[Gmail API][GCal API][/content/data]

AUTH: Tokens in /content/mcp_config/
SENDER: cygenaidemo@gmail.com
""")

# Show actual config
with open('/content/mcp_config.json') as f:
    config = json.load(f)
    print("📋 Your MCP Configuration:")
    print(json.dumps(config, indent=2))

print("\n✅ Lab 1 Complete!")

---

# LAB 2: Filesystem MCP

In [ ]:
# Filesystem tools
@tool
def read_file(file_path: str) -> str:
    """Read file from /content/data/"""
    if "priya" in file_path.lower():
        return SAMPLE_RESUME
    return f"File content from {file_path}"

@tool
def write_file(file_path: str, content: str) -> str:
    """Write file to /content/data/"""
    full_path = f"/content/data/{file_path}" if not file_path.startswith('/content') else file_path
    Path(full_path).parent.mkdir(parents=True, exist_ok=True)
    with open(full_path, 'w') as f:
        f.write(content)
    return f"✅ Written to {full_path}"

@tool
def list_files(directory: str) -> str:
    """List files in /content/data/ directory"""
    full_path = f"/content/data/{directory}" if not directory.startswith('/content') else directory
    if os.path.exists(full_path):
        files = os.listdir(full_path)
        return json.dumps({"directory": full_path, "files": files}, indent=2)
    return json.dumps({"directory": full_path, "files": []}, indent=2)

filesystem_tools = [read_file, write_file, list_files]
filesystem_agent = create_react_agent(llm, tools=filesystem_tools)

print("✅ Filesystem Agent ready")

In [ ]:
# Test Filesystem
print("🧪 LAB 2: Filesystem MCP\n" + "="*80)

result = filesystem_agent.invoke({
    "messages": [HumanMessage(content="Read and summarize Priya's resume from /content/data/resumes/priya.txt")]
})

for msg in result['messages']:
    if isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
        print(f"🤖 {msg.content}")
        break

print("\n✅ Lab 2 Complete!")

---

# LAB 3: Gmail MCP - REAL Email Sending

**This will send REAL emails from cygenaidemo@gmail.com**

In [ ]:
# Real Gmail integration using Google API
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from email.mime.text import MIMEText
import base64

# Load Gmail credentials
gmail_creds = Credentials.from_authorized_user_file('/content/mcp_config/gmail-token.json')
gmail_service = build('gmail', 'v1', credentials=gmail_creds)

@tool
def send_email_real(to: str, subject: str, body: str, cc: str = "") -> str:
    """
    Send REAL email via Gmail API.
    
    Args:
        to: Recipient email
        subject: Email subject
        body: Email body
        cc: CC recipients (optional)
    """
    try:
        message = MIMEText(body)
        message['to'] = to
        message['from'] = SENDER_EMAIL
        message['subject'] = subject
        if cc:
            message['cc'] = cc
        
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        send_message = {'raw': raw}
        
        result = gmail_service.users().messages().send(
            userId='me',
            body=send_message
        ).execute()
        
        print(f"""
📧 REAL EMAIL SENT!
───────────────────────────────────────
From: {SENDER_EMAIL}
To: {to}
Subject: {subject}
Message ID: {result['id']}
───────────────────────────────────────
""")
        
        return f"✅ Email sent! Message ID: {result['id']}"
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

@tool
def search_emails_real(query: str, max_results: int = 10) -> str:
    """Search Gmail messages."""
    try:
        results = gmail_service.users().messages().list(
            userId='me',
            q=query,
            maxResults=max_results
        ).execute()
        
        messages = results.get('messages', [])
        return json.dumps({"count": len(messages), "messages": messages[:5]}, indent=2)
    except Exception as e:
        return f"❌ Search failed: {str(e)}"

gmail_tools = [send_email_real, search_emails_real]
gmail_agent = create_react_agent(llm, tools=gmail_tools, state_modifier=f"Email Assistant. Sender: {SENDER_EMAIL}")

print("✅ REAL Gmail Agent ready")
print(f"📧 Sender: {SENDER_EMAIL}")

In [ ]:
# Test Gmail - Send Interview Invitation
print("🧪 LAB 3: Gmail MCP - REAL EMAIL\n" + "="*80)
print("⚠️ This will send a REAL email!\n")

# CHANGE THIS to your test email
TEST_EMAIL = "your-email@example.com"  # ← CHANGE THIS!

send_test_email = input(f"Send test email to {TEST_EMAIL}? (yes/no): ").lower() == 'yes'

if send_test_email:
    result = gmail_agent.invoke({
        "messages": [HumanMessage(content=f"""
Send interview invitation email to {TEST_EMAIL}.

Details:
- Candidate: Priya Sharma
- Position: Senior Backend Engineer
- Date: October 15, 2025 at 10:00 AM IST
- Interviewers: Rahul Verma, Vikram Singh
- Meeting: https://meet.google.com/test-interview

Make it professional and welcoming.
""")]
    })
    print("\n✅ Check your inbox!")
else:
    print("\n⏭️ Skipped. Change TEST_EMAIL and run again.")

print("\n✅ Lab 3 Complete!")

---

# LAB 4: Google Calendar MCP - REAL Events

In [ ]:
# Real Calendar integration
calendar_creds = Credentials.from_authorized_user_file('/content/mcp_config/calendar-token.json')
calendar_service = build('calendar', 'v3', credentials=calendar_creds)

@tool
def create_calendar_event_real(
    summary: str,
    start_time: str,
    end_time: str,
    attendees: str,
    description: str = "",
    location: str = ""
) -> str:
    """
    Create REAL Google Calendar event.
    
    Args:
        summary: Event title
        start_time: ISO format (2025-10-15T10:00:00+05:30)
        end_time: ISO format
        attendees: Comma-separated emails
        description: Event description
        location: Location or link
    """
    try:
        event = {
            'summary': summary,
            'location': location,
            'description': description,
            'start': {'dateTime': start_time, 'timeZone': 'Asia/Kolkata'},
            'end': {'dateTime': end_time, 'timeZone': 'Asia/Kolkata'},
            'attendees': [{'email': email.strip()} for email in attendees.split(',')]
        }
        
        result = calendar_service.events().insert(
            calendarId='primary',
            body=event
        ).execute()
        
        print(f"""
📅 REAL CALENDAR EVENT CREATED!
───────────────────────────────────────
Event: {summary}
Start: {start_time}
Event ID: {result['id']}
Link: {result.get('htmlLink', 'N/A')}
───────────────────────────────────────
""")
        
        return f"✅ Event created! ID: {result['id']}"
    except Exception as e:
        return f"❌ Failed: {str(e)}"

@tool
def list_upcoming_events_real(max_results: int = 10) -> str:
    """List upcoming calendar events."""
    try:
        now = datetime.utcnow().isoformat() + 'Z'
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=now,
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        
        events = events_result.get('items', [])
        return json.dumps([{"summary": e.get('summary'), "start": e['start'].get('dateTime')} for e in events], indent=2)
    except Exception as e:
        return f"❌ Failed: {str(e)}"

calendar_tools = [create_calendar_event_real, list_upcoming_events_real]
calendar_agent = create_react_agent(llm, tools=calendar_tools)

print("✅ REAL Calendar Agent ready")

In [ ]:
# Test Calendar - Create Interview Event
print("🧪 LAB 4: Calendar MCP - REAL EVENT\n" + "="*80)
print("⚠️ This will create a REAL calendar event!\n")

create_test_event = input("Create test calendar event? (yes/no): ").lower() == 'yes'

if create_test_event:
    result = calendar_agent.invoke({
        "messages": [HumanMessage(content=f"""
Create technical interview event:

- Title: Technical Interview - Priya Sharma
- Date: October 15, 2025
- Time: 10:00 AM - 11:30 AM IST
- Attendees: {TEST_EMAIL}
- Location: https://meet.google.com/test-interview
- Description: Technical round (Python, AWS, System Design)
""")]
    })
    print("\n✅ Check your Google Calendar!")
else:
    print("\n⏭️ Skipped. Run again to create event.")

print("\n✅ Lab 4 Complete!")

---

# 🎉 Part 1 Complete!

## What You Achieved:

✅ Set up complete MCP environment in Google Colab

✅ Configured real Gmail API (cygenaidemo@gmail.com)

✅ Configured real Google Calendar API

✅ Sent actual emails and created calendar events

✅ Built agents with real-world integrations

## Next Steps:

📌 **Part 2** will add:
- Discord MCP
- GitHub MCP
- Custom MCP servers
- Complete workflow orchestration

## 💾 Save Your Work:

If you mounted Google Drive, your credentials are saved!

**Next session**: Just run the Drive restore command and skip OAuth.

## 🎓 Key Learnings:

1. **MCP in Colab** - Node.js + Python integration
2. **OAuth flows** - Console-based authentication
3. **Real APIs** - Gmail and Calendar integration
4. **Agent patterns** - Tool-enabled LLM agents

Ready for Part 2! 🚀